<a id="top"></a>
# RF Classifier Sample Application

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **rf_FP32.xml + rf_FP32.bin** - The xml file of the model (or rf_FP16.xml + rf_FP16.bin) 
    - **rf_classifier.py** - Base Python* code to perform inference
    - **RF_TUTORIAL.ipynb** - This Jupyter* Notebook
    - **rf_classy.sh** - Deployment shell script
    - **RML2016.10a_dict.pkl** - The dataset on which to test on, in .pkl format
    
It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


## Introduction

This sample application demonstrates how inference on RF data may be performed using Intel® hardware and software tools. This solution uses a model derived from the following author: 

[@article{convnetmodrec,
  title={Convolutional Radio Modulation Recognition Networks},
  author={O'Shea, Timothy J and Corgan, Johnathan and Clancy, T. Charles},
  journal={arXiv preprint arXiv:1602.04105},
  year={2016}
}](https://github.com/radioML/examples/blob/master/modulation_recognition/RML2016.10a_VTCNN2_example.ipynb)

[@article{rml_datasets,
  title={Radio Machine Learning Dataset Generation with GNU Radio},
  author={O'Shea, Timothy J and West, Nathan},
  journal={Proceedings of the 6th GNU Radio Conference},
  year={2016}
}](https://github.com/radioML/examples/blob/master/modulation_recognition/RML2016.10a_VTCNN2_example.ipynb) 

Additionally, the RML2016.10a dataset is used for this work (https://www.deepsig.ai/datasets).


### Key concepts
This sample application includes an example for the following:
- Application:
  - Timestream data as input is supported in OpenVINO
  - Inference can be performed on large datasets
  - Inference can be run in multiple batches
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


## RF Classification

The RF modulation classifier uses the Intel® Distribution of OpenVINO™ toolkit to classify 11 different modulations of the RF data, as described by the RADIOML 2016.10A dataset. The following steps are taken to take output

1. Convert the model to IR to be used for inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the RF application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified timeframe data.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook.

The application and inference code for this sample is already implemented in the two Python* files, as described above.

The following sections will guide you through configuring and running the RF modulation classifier

### Configuration
The following sections describe all the necessary configuration to run the RF classifier application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 rf_classifier.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`rf_classifier.py`](./rf_classifier.py) are:
- **-m** - Path to the _.xml_ IR file (created using [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)) for the inference model
- **-o** - The path to where the output video file will be stored
- **-d** - Device type to use to run inference (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)

Note: Other examples will have a -i argument for inputs because the desired files to input will be in your file directory. The dataset we wish to test on contains 60000 datapoints, but we can't iterate through them one-by-one in just the files. We will unpack the dataset in the classification code itself, so there isn't an excessive use of memory.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.). 
The Intel® Distribution of OpenVINO™ toolkit also includes the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) utility  to download some common inference models from the [Open Model Zoo](https://github.com/opencv/open_model_zoo). 


### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

Use the classification script below, which imports in the dataset for testing, formats it, and performs inference using the OpenVINO™ IECore. Make sure the RML2016.10a_dict.pkl file is in the same directory, or change the path to it in the code below

In [ ]:
import os
import logging as log

from PIL import Image
import numpy as np
import cv2
import sys
import os
from argparse import ArgumentParser
from qarpo.demoutils import *
import applicationMetricWriter
from time import time

# Setup the dataset
import _pickle as cPickle

# Make sure this file is in your current directory!
with open("RML2016.10a_dict.pkl", 'rb') as f:
    Xd = cPickle.load(f, encoding="latin1") 

#Setup the input data: snr = signal to noise ratio, mod = modulation scheme
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])

#Let X be our array of modulations
X = []
labels = []

for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  labels.append((mod,snr))
X = np.vstack(X)

#Setup inference engine
try:
    from openvino import inference_engine as ie
    from openvino.inference_engine import IENetwork, IECore, IEPlugin
    
except Exception as e:
    exception_type = type(e).__name__
    print("The following error happened while importing Python API module:\n[ {} ] {}".format(exception_type, e))
    sys.exit(1)

def build_argparser():
    parser = ArgumentParser()
    parser.add_argument("-m", "--model", help="Path to an .xml file with a trained model.", required=True, type=str)
    parser.add_argument("-d", "--device",
                        help="Specify the target device to infer on; CPU, GPU, FPGA or MYRIAD is acceptable. Sample "
                             "will look for a suitable plugin for device specified (CPU by default)", default="CPU",
                        type=str)
    parser.add_argument("-ni", "--number_iter", help="Number of inference iterations", default=1, type=int)
    parser.add_argument("-l", "--cpu_extension",
                        help="MKLDNN (CPU)-targeted custom layers.Absolute path to a shared library with the kernels "
                             "impl.", type=str, default=None)
    parser.add_argument("-pp", "--plugin_dir", help="Path to a plugin folder", type=str, default=None)
    parser.add_argument("--num_threads", default=88, type=int)
    parser.add_argument("-nt", "--number_top", help="Number of top results", default=11, type=int)
    parser.add_argument("-p", "--out_dir", help="Optional. The path where result files and logs will be stored",
                      required=False, default="./results", type=str)
    parser.add_argument("-o", "--out_prefix", 
                      help="Optional. The file name prefix in the output_directory where results will be stored", 
                      required=False, default="out_", type=str)
    parser.add_argument("-g", "--log_prefix", 
                      help="Optional. The file name prefix in the output directory for log files",
                      required=False, default="log_", type=str)

    return parser

def main():
    # Run inference
    job_id = os.getenv("PBS_JOBID")    
    
    # Plugin initialization for specified device and load extensions library if specified.
    args = build_argparser().parse_args()
    model_xml = args.model
    model_bin = os.path.splitext(model_xml)[0] + ".bin"

    # Set up logging to a file
    logpath = os.path.join(os.path.dirname(__file__), ".log")
    log.basicConfig(level=log.INFO,
                    format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s",
                    filename=logpath,
                    filemode="w" )
    try:
        job_id = os.environ['PBS_JOBID']
        infer_file = os.path.join(args.out_dir,'i_progress_'+str(job_id)+'.txt')
    except Exception as e:
        log.warning(e)
        log.warning("job_id: {}".format(job_id))
    
    # Setup additional logging to console
    console = log.StreamHandler()
    console.setLevel(log.INFO)
    formatter = log.Formatter("[ %(levelname)s ] %(message)s")
    console.setFormatter(formatter)
    log.getLogger("").addHandler(console)
    
    # Plugin initialization for specified device and load extensions library if specified
    log.info("Initializing plugin for {} device...".format(args.device))
    ie = IECore()
   
    if args.cpu_extension and 'CPU' in args.device:
        log.info("Loading plugins for {} device...".format(args.device))
        ie.add_extension(args.cpu_extension, "CPU")

    # Read IR
    log.info("Reading IR...")
    net = ie.read_network(model=model_xml, weights=model_bin)

    if "CPU" in args.device:
        supported_layers = ie.query_network(net, "CPU")
        not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
        if len(not_supported_layers) != 0:
            log.warning("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(args.device, ', '.join(not_supported_layers)))
            log.warning("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                      "or --cpu_extension command line argument")
            sys.exit(1)

    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
   
    # Load network to the plugin
    log.info("Loading model to the plugin")
    exec_net = ie.load_network(network=net, device_name=args.device)
     
    #CLASSES = 11
    class_names = ["8PSK", "AM-DSB", "AM-SSB", "BPSK", "CPFSK", "GFSK", "PAM4", "QAM16", "QAM64", "QPSK", "WBFM"]

    print("Preparing input blobs")
    
    # We define the batch size as x for easier use throughout
    
    # Note that if you want to have a different batch size, you would have to create different IR (see the Juypter Notebook
    # for more information on this)
    x = 110
    print("Batch size is {}".format(x))
        
    correct = 0
    wrong = 0
    total_inference = 0
    topnum = 0
    j = 0
    
    def run_it(start):
        #Setup an array to run inference on
        modulations = np.ndarray(shape=(x, 128, 1, 2))
        
        #Fill up the input array for this batch
        stop = start + x
        i = 0
        
        for item in X[start:stop]:
            modulations[i] = item.reshape([1,2,128]).transpose(2, 0, 1)
            i += 1

        # Loading model to the plugin    
        # Start inference
        infer_time = []

        t0 = time()
        res = exec_net.infer(inputs={input_blob: modulations})
        infer_time.append((time()-t0)*1000)

        # Processing output blob
        res = res[out_blob]
        
        #Check results for accuracy
        
        nonlocal correct
        nonlocal wrong
        nonlocal j
        
        # Function for calculating the amount correct, up to a certain "num"
        # For example, for num = 3, would return topnum, representing the amount of 
        # times the correct label was one of the top 3 probabilities predicted
        def topnum_accuracy(num):
            nonlocal topnum
            
            for i in range(num):
                det_label = class_names[top_ind[i]]

                if det_label == labels[j][0]:
                    topnum += 1
                    return
        
        #Automatically calculates the accuracy for top 1 predictions
        for i, probs in enumerate(res):
            probs = np.squeeze(probs)
            top_ind = np.argsort(probs)[-args.number_top:][::-1]
           
            det_label = class_names[top_ind[0]]
            
            if det_label == labels[j][0]:
                correct = correct + 1
            else:
                wrong = wrong + 1        
            
            #Default to calculating top-3 accuracy
            topnum_accuracy(3)
            
            j = j + 1        
            
        nonlocal total_inference
        total_inference += np.sum(np.asarray(infer_time))
        
        
    #Iterate through the whole dataset    
    num_batches = X.shape[0]//x
   
    k = 0
    print("Running inference: Batch 1")
    
    #Run it on the dataset
    for i in range(num_batches):
        if (i + 1) % 100 == 0:
            print("Running inference: Batch " + str(i + 1))
        run_it(k)
        k += x
         
    # Print results    
    print("Correct " + str(correct))
    print("Wrong " + str(wrong))
    print("Accuracy: " + str(correct/(correct + wrong)))
    print("Top " + str(topnum) + " Correct: " + str(topnum))
    print("Top " + str(topnum) + " Accuracy: " + str(topnum/(correct + wrong)))

    print("Average running time of one batch: {} ms".format(total_inference/num_batches))
    print("Total running time of inference: {} ms" .format(total_inference))
    print("Throughput: {} FPS".format((1000*args.number_iter*x*num_batches)/total_inference))
    
    import platform
    platform.processor()
    print("Processor: " + platform.processor())
    print("\n")
        
    #Cleanup
    del net
    del exec_net

if __name__ == '__main__':
    sys.exit(main() or 0) 

#### Aside: Changing the Batch size

In the above code, you may notice that the batch size is a constant "x", where x = 110. Changing this value will put errors into the code, because when the model was generated into IR, the "input_shape" paramater was "[110, 1, 2, 128]". If you wish to run the code with a different batch size, you will need to create new IR from the .onnx model, and specify the first value of the input_shape paramter to change the batch size. Then, in the code, you simply change x to the new value. 

#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
rf_classy.sh <output_directory> <device> <fp_precision> <input_file> <threshold>
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*device*> - Hardware device to use (e.g. CPU, GPU, etc.)
- <*data_type*> - Which floating point precision inference model to use (FP32 or FP16)

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Do additional setup and configuration when running inference on an FPGA hardware device
- Choose the appropriate inference model IR file for the specified <*fp_precision*>
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `rf_classy.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `rf_classy.sh`.

In [1]:
# Store input arguments: <output_directory> <device> <fp_precision> <input_file>
cd $HOME/rf
OUTPUT_FILE=$1
DEVICE=$2
DTYPE=$3

# The default path for the job is the user's home directory,
#  change directory to where the files are.

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Check for special setup steps depending upon device to be used
if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs - Updated for OpenVINO 2020.3
    export AOCL_BOARD_PACKAGE_ROOT=/opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
  #  export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
  
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/2020-4_PL2_FP11_AlexNet_GoogleNet_Generic.aocx

fi

# if [ "$DTYPE" = "FP16" ]; then
#     export MODEL=$HOME/rf/FP16rfoink.xml
    
# else
#     export MODEL=$HOME/rf/rfoink.xml
    
fi

python3 $HOME/rf/rf_tutorial/large_rf.py -m $MODEL -d $DEVICE

# Set inference model IR files using specified precision
# Run the classifier  code


SyntaxError: invalid syntax (<ipython-input-1-6feaa4a8de15>, line 2)

#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit rd_classy.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [1]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

     17 idc001skl,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe
     16 idc002mx8,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-r,iei-mustang-v100-mx8
     13 idc003a10,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-f,iei-mustang-f100-a10
     11 idc004nc2,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,ncs,intel-ncs2
      5 idc006kbl,compnode,iei,tank-870,intel-core,i5-7500t,kaby-lake,intel-hd-630,ram8gb,net1gbe
      4 idc007xv5,compnode,iei,tank-870,intel-xeon,e3-1268l-v5,skylake,intel-hd-p530,ram32gb,net1gbe
      7 idc008u2g,compnode,up-squared,grove,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe,ncs,intel-ncs2
      1 idc009jkl,compnode,jwip,intel-core,i5-7500,kaby-lake,intel-hd-630,ram8gb,net1gbe
      1 idc010jal,compnode,jwip,intel-celeron,j3355,apollo-lake,intel-hd-500,ram4gb,net1gbe
      1 idc011ark2250s,compnode,advantech,int

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submitting a job on this notebook's node

You can run the inference on the notebook the computer is connected to right now.

In [4]:
!python3 rf_classifier.py -m rf_FP16.xml

[ INFO ] Initializing plugin for CPU device...
[ INFO ] Reading IR...
rf_classifier.py:123: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  input_blob = next(iter(net.inputs))
[ INFO ] Loading model to the plugin
Preparing input blobs
Batch size is 110
Running inference: Batch 1
Running inference: Batch 100
Running inference: Batch 200
Running inference: Batch 300
Running inference: Batch 400
Running inference: Batch 500
Running inference: Batch 600
Running inference: Batch 700
Running inference: Batch 800
Running inference: Batch 900
Running inference: Batch 1000
Running inference: Batch 1100
Running inference: Batch 1200
Running inference: Batch 1300
Running inference: Batch 1400
Running inference: Batch 1500
Running inference: Batch 1600
Running inference: Batch 1700
Running inference: Batch 1800
Running inference: Batch 1900
Run

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [16]:
#Submit job to the queue
job_id_core = !qsub rf_classy.sh -l nodes=1:i5-6500te -F ". CPU FP32" -N rf_core
print(job_id_core[0])
#Progress indicators

44971.v-qsvr-1.devcloud-edge


##### Submit to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [55]:
#Submit job to the queue
job_id_xeon = !qsub rf_classy.sh  -l nodes=1:e3-1268l-v5 -F ". CPU FP32" -N rf_xeon 
print(job_id_xeon[0])

41988.v-qsvr-1.devcloud-edge


##### Submit to an edge compute node with Intel® Core CPU and using the integrated Intel® GPU
In the cell below, we submit a job to an edge node with an [Intel® Core i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html). The inference workload will run on the Intel® HD Graphics 530 GPU integrated with the CPU.

In [71]:
#Submit job to the queue
job_id_gpu = !qsub rf_classy.sh -l nodes=1:i5-6500te:intel-hd-530 -F ". GPU FP32" -N rf_gpu 
print(job_id_gpu[0])

41994.v-qsvr-1.devcloud-edge


### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [3]:
liveQstat()

NameError: name 'liveQstat' is not defined

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>